In [3]:
# Check if the code is running in Google Colab
try:
    from google.colab import files

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # If in Google Colab, use file upload functionality
    uploaded = files.upload()
    db_filename = list(uploaded.keys())[0]
else:
    # If not in Google Colab, load the file directly from the local directory
     db_filename = 'C:/Users/sahoo/Downloads/archive/valorant.sqlite'

In [5]:
import sqlite3
import pandas as pd

In [6]:
conn = sqlite3.connect(db_filename)

In [7]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Tables in the database:")
print(tables)

Tables in the database:
              name
0          Matches
1            Games
2      Game_Rounds
3  Game_Scoreboard


In [8]:
for table_name in tables['name']:
    df = pd.read_sql(f'SELECT * FROM {table_name}', conn)

    if IN_COLAB:  # if in colab save in root directory
        csv_filename = f"{table_name}.csv"
    else:  # if not in colab save in subdirectory /data/csv
        csv_filename = f"C:/Users/sahoo/Downloads/archive/{table_name}.csv"

    df.to_csv(csv_filename, index=False)

    print(f"Table {table_name} has been saved as {csv_filename}")

Table Matches has been saved as C:/Users/sahoo/Downloads/archive/Matches.csv
Table Games has been saved as C:/Users/sahoo/Downloads/archive/Games.csv
Table Game_Rounds has been saved as C:/Users/sahoo/Downloads/archive/Game_Rounds.csv
Table Game_Scoreboard has been saved as C:/Users/sahoo/Downloads/archive/Game_Scoreboard.csv


In [9]:
# Check if the code is running in Google Colab
# and set the directory variable to its corresponding value
if IN_COLAB:
    directory = "."
else:
    directory = "C:/Users/sahoo/Downloads/archive"

In [10]:
import os

csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
print("Generated CSV files:")
print(csv_files)

Generated CSV files:
['Games.csv', 'Games_Cleaned.csv', 'Games_Normalized.csv', 'Game_Rounds.csv', 'Game_Rounds_Cleaned.csv', 'Game_Scoreboard.csv', 'Game_Scoreboard_Cleaned.csv', 'Game_Scoreboard_Normalized.csv', 'Matches.csv', 'Matches_Cleaned.csv', 'Matches_Normalized.csv', 'Role_Scores_By_Agent.csv']


In [11]:
if IN_COLAB:
    from google.colab import files

    # Download each CSV file
    for file in csv_files:
        files.download(file)

In [12]:
import pandas as pd

file_names = ['Game_Scoreboard.csv', 'Game_Rounds.csv', 'Matches.csv', 'Games.csv']

game_scoreboard_df = pd.read_csv(f"{directory}/Game_Scoreboard.csv")
game_rounds_df = pd.read_csv(f"{directory}/Game_Rounds.csv")
matches_df = pd.read_csv(f"{directory}/Matches.csv")
games_df = pd.read_csv(f"{directory}/Games.csv")

print("Game_Scoreboard DataFrame:")
print(game_scoreboard_df.head())

print("\nGame_Rounds DataFrame:")
print(game_rounds_df.head())

print("\nMatches DataFrame:")
print(matches_df.head())

print("\nGames DataFrame:")
print(games_df.head())

Game_Scoreboard DataFrame:
   GameID  PlayerID PlayerName TeamAbbreviation    Agent    ACS  Kills  \
0   60894    8419.0     Reduxx             Boos     jett  313.0   24.0   
1   60894     466.0     ChurmZ             Boos  chamber  227.0   16.0   
2   60894    3712.0   diaamond             Boos     sova  226.0   17.0   
3   60894    5099.0     Boltzy             Boos    viper  218.0   17.0   
4   60894    3983.0     Virtyy             Boos     skye   80.0    5.0   

   Deaths  Assists  PlusMinus  ...  Num_4Ks  Num_5Ks  OnevOne  OnevTwo  \
0    10.0      3.0       14.0  ...      2.0      0.0      1.0      0.0   
1    10.0      7.0        6.0  ...      0.0      0.0      0.0      0.0   
2     9.0      8.0        8.0  ...      0.0      0.0      1.0      0.0   
3    12.0      2.0        5.0  ...      0.0      0.0      1.0      0.0   
4    13.0      3.0       -8.0  ...      0.0      0.0      0.0      0.0   

   OnevThree  OnevFour  OnevFive  Econ  Plants  Defuses  
0        0.0       0.0   

In [13]:
def clean_dataframe(df, threshold=0.2):
    threshold_value = int(df.shape[1] * threshold)
    df = df[df.isnull().sum(axis=1) <= threshold_value]
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df.loc[:, column] = df.loc[:, column].fillna(df[column].mean())
        else:
            df.loc[:, column] = df.loc[:, column].fillna('Unknown')
    df = df.drop_duplicates()
    return df

game_scoreboard_cleaned = clean_dataframe(game_scoreboard_df, threshold=0.2)
game_rounds_cleaned = clean_dataframe(game_rounds_df, threshold=0.2)
matches_cleaned = clean_dataframe(matches_df, threshold=0.2)
games_cleaned = clean_dataframe(games_df, threshold=0.2)

# show number of rows in games_scoreboard_df
print(f"Number of rows in Game_Scoreboard DataFrame: {game_scoreboard_df.shape[0]}")
print(f"Number of rows in Game_Rounds DataFrame: {game_rounds_df.shape[0]}")
print(f"Number of rows in Matches DataFrame: {matches_df.shape[0]}")
print(f"Number of rows in Games DataFrame: {games_df.shape[0]}")

print()

print(f"Number of rows in cleaned Game_Scoreboard DataFrame: {game_scoreboard_cleaned.shape[0]}")
print(f"Number of rows in cleaned Game_Rounds DataFrame: {game_rounds_cleaned.shape[0]}")
print(f"Number of rows in cleaned Matches DataFrame: {matches_cleaned.shape[0]}")
print(f"Number of rows in cleaned Games DataFrame: {games_cleaned.shape[0]}")

print()

# print percentage of dropped rows in each data frame
print(f"Percentage of dropped rows in Game_Scoreboard DataFrame: {(1 - game_scoreboard_cleaned.shape[0] / game_scoreboard_df.shape[0]) * 100:.2f}%")
print(f"Percentage of dropped rows in Game_Rounds DataFrame: {(1 - game_rounds_cleaned.shape[0] / game_rounds_df.shape[0]) * 100:.2f}%")
print(f"Percentage of dropped rows in Matches DataFrame: {(1 - matches_cleaned.shape[0] / matches_df.shape[0]) * 100:.2f}%")
print(f"Percentage of dropped rows in Games DataFrame: {(1 - games_cleaned.shape[0] / games_df.shape[0]) * 100:.2f}%")

game_scoreboard_cleaned.to_csv(f"{directory}/Game_Scoreboard_Cleaned.csv", index=False)
game_rounds_cleaned.to_csv(f"{directory}/Game_Rounds_Cleaned.csv", index=False)
matches_cleaned.to_csv(f"{directory}/Matches_Cleaned.csv", index=False)
games_cleaned.to_csv(f"{directory}/Games_Cleaned.csv", index=False)


C:\Users\sahoo\AppData\Local\Temp\ipykernel_14752\684217951.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, column] = df.loc[:, column].fillna(df[column].mean())
C:\Users\sahoo\AppData\Local\Temp\ipykernel_14752\684217951.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, column] = df.loc[:, column].fillna('Unknown')


Number of rows in Game_Scoreboard DataFrame: 157939
Number of rows in Game_Rounds DataFrame: 15531
Number of rows in Matches DataFrame: 7818
Number of rows in Games DataFrame: 15888

Number of rows in cleaned Game_Scoreboard DataFrame: 147744
Number of rows in cleaned Game_Rounds DataFrame: 14854
Number of rows in cleaned Matches DataFrame: 7818
Number of rows in cleaned Games DataFrame: 14854

Percentage of dropped rows in Game_Scoreboard DataFrame: 6.46%
Percentage of dropped rows in Game_Rounds DataFrame: 4.36%
Percentage of dropped rows in Matches DataFrame: 0.00%
Percentage of dropped rows in Games DataFrame: 6.51%


In [14]:
import pandas as pd

game_scoreboard_cleaned_df = pd.DataFrame(game_scoreboard_cleaned)
game_rounds_cleaned_df = pd.DataFrame(game_rounds_cleaned)
matches_cleaned_df = pd.DataFrame(matches_cleaned)
games_cleaned_df = pd.DataFrame(games_cleaned)


print("Cleaned Game_Scoreboard DataFrame:")
print(game_scoreboard_cleaned_df.head())

print("\nCleaned Game_Rounds DataFrame:")
print(game_rounds_cleaned_df.head())

print("\nCleaned Matches DataFrame:")
print(matches_cleaned_df.head())

print("\nCleaned Games DataFrame:")
print(games_cleaned_df.head())

Cleaned Game_Scoreboard DataFrame:
   GameID  PlayerID PlayerName TeamAbbreviation    Agent    ACS  Kills  \
0   60894    8419.0     Reduxx             Boos     jett  313.0   24.0   
1   60894     466.0     ChurmZ             Boos  chamber  227.0   16.0   
2   60894    3712.0   diaamond             Boos     sova  226.0   17.0   
3   60894    5099.0     Boltzy             Boos    viper  218.0   17.0   
4   60894    3983.0     Virtyy             Boos     skye   80.0    5.0   

   Deaths  Assists  PlusMinus  ...  Num_4Ks  Num_5Ks  OnevOne  OnevTwo  \
0    10.0      3.0       14.0  ...      2.0      0.0      1.0      0.0   
1    10.0      7.0        6.0  ...      0.0      0.0      0.0      0.0   
2     9.0      8.0        8.0  ...      0.0      0.0      1.0      0.0   
3    12.0      2.0        5.0  ...      0.0      0.0      1.0      0.0   
4    13.0      3.0       -8.0  ...      0.0      0.0      0.0      0.0   

   OnevThree  OnevFour  OnevFive  Econ  Plants  Defuses  
0        0.0     

New Code Below

In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Function to normalize the data
def normalize_dataframe(df):
    scaler = MinMaxScaler()
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    return df

# Normalize the data
game_rounds_normalized = normalize_dataframe(game_rounds_cleaned_df)
games_normalized = normalize_dataframe(games_cleaned_df)
matches_normalized = normalize_dataframe(matches_cleaned_df)
game_scoreboard_normalized = normalize_dataframe(game_scoreboard_cleaned_df)

# Save the normalized data to new CSV files
game_rounds_normalized.to_csv(f"{directory}/Game_Rounds_Normalized.cs", index=False)
games_normalized.to_csv(f"{directory}/Games_Normalized.csv", index=False)
matches_normalized.to_csv(f"{directory}/Matches_Normalized.csv", index=False)
game_scoreboard_normalized.to_csv(f"{directory}/Game_Scoreboard_Normalized.csv", index=False)

# Output to confirm completion
print(f"Game_Rounds_Normalized shape: {game_rounds_normalized.shape}")
print(f"Games_Normalized shape: {games_normalized.shape}")
print(f"Matches_Normalized shape: {matches_normalized.shape}")
print(f"Game_Scoreboard_Normalized shape: {game_scoreboard_normalized.shape}")

Game_Rounds_Normalized shape: (14854, 4)
Games_Normalized shape: (14854, 36)
Matches_Normalized shape: (7818, 12)
Game_Scoreboard_Normalized shape: (147744, 28)


In [16]:
 game_scoreboard_filename = 'C:/Users/sahoo/Downloads/archive/Game_Scoreboard_Normalized.csv'
game_scoreboard_normalized_df = pd.read_csv(game_scoreboard_filename)
agent_averages_df = game_scoreboard_normalized_df.groupby('Agent').mean().reset_index()

def calculate_role_scores(df):
    role_scores = {
        'Agent': [],
        'Entry Fragger': [],
        'Support': [],
        'Sniper': [],
        'Lurker': [],
        'IGL': [],
        'Anchor': []
    }
    
    for _, row in df.iterrows():
        scores = {
            'Entry Fragger': row['FirstKills'] - row['FirstDeaths'] + row['FKFD_PlusMinus'] + row['ACS'] + row['ADR'],
            'Support': row['Assists'] + row['Plants'] + row['Defuses'] + (row['Num_2Ks'] + row['Num_3Ks'] + row['Num_4Ks'] + row['Num_5Ks']) / 4 + row['OnevOne'] + row['OnevTwo'],
            'Sniper': row['ACS'] + row['ADR'] + row['HS_Percent'] + row['PlusMinus'],
            'Lurker': row['KAST_Percent'] + row['PlusMinus'] + (row['Num_2Ks'] + row['Num_3Ks'] + row['Num_4Ks'] + row['Num_5Ks']) / 4 + row['Econ'],
            'IGL': (row['ACS'] + row['ADR'] + row['Kills'] + row['Assists'] + row['PlusMinus'] + row['KAST_Percent'] + row['Plants'] + row['Defuses']) / 8,
            'Anchor': row['ADR'] + row['KAST_Percent'] + row['Defuses'] + row['PlusMinus'] + row['OnevOne'] + row['OnevTwo']
        }
        
        role_scores['Agent'].append(row['Agent'])
        role_scores['Entry Fragger'].append(scores['Entry Fragger'])
        role_scores['Support'].append(scores['Support'])
        role_scores['Sniper'].append(scores['Sniper'])
        role_scores['Lurker'].append(scores['Lurker'])
        role_scores['IGL'].append(scores['IGL'])
        role_scores['Anchor'].append(scores['Anchor'])

    return pd.DataFrame(role_scores)
role_scores_df = calculate_role_scores(agent_averages_df)
 role_scores_df.to_csv('C:/Users/sahoo/Downloads/archive/Role_Scores_By_Agent.csv', index=False)



In [17]:
print(f"Role_Scores_By_Agent shape: {role_scores_df.shape}")

Role_Scores_By_Agent shape: (17, 7)


In [18]:
 print(role_scores_df.head())

       Agent  Entry Fragger   Support    Sniper    Lurker       IGL    Anchor
0      astra       0.998731  0.541417  1.253848  1.296209  0.284736  1.475342
1     breach       0.931147  0.649888  1.149647  1.242299  0.287544  1.428197
2  brimstone       0.981145  0.570483  1.207209  1.276860  0.285960  1.467763
3    chamber       1.104931  0.363867  1.372691  1.354727  0.277809  1.539263
4     cypher       0.977040  0.421419  1.228481  1.278566  0.267858  1.449287
